In [59]:
library(dplyr)
library(caret)
library(lattice)
library(ggplot2)
library(qcc)
library(stats)
library(Hmisc)
library(car)
library(leaps)
library(outliers)
library(kknn)
library(knitr)
library(psych)
library(corrplot)
library(tidyverse)
library(cli)
library(devtools)
library(tree)
library(rpart)
library(pROC)
#library(pR2)
#library(multinom)

library(lattice)
library(ggplot2)

library(comprehenr)
library(gridExtra)
library(MASS)

setwd("~/Downloads/Datasets/7406/Project")

white <- read.csv(file = "winequality_white.csv", head = TRUE, sep=";")

#white$quality <- as.factor(white$quality)
str(white)
summary(white)
head(white)



'data.frame':	4898 obs. of  12 variables:
 $ fixed.acidity       : num  7 6.3 8.1 7.2 7.2 8.1 6.2 7 6.3 8.1 ...
 $ volatile.acidity    : num  0.27 0.3 0.28 0.23 0.23 0.28 0.32 0.27 0.3 0.22 ...
 $ citric.acid         : num  0.36 0.34 0.4 0.32 0.32 0.4 0.16 0.36 0.34 0.43 ...
 $ residual.sugar      : num  20.7 1.6 6.9 8.5 8.5 6.9 7 20.7 1.6 1.5 ...
 $ chlorides           : num  0.045 0.049 0.05 0.058 0.058 0.05 0.045 0.045 0.049 0.044 ...
 $ free.sulfur.dioxide : num  45 14 30 47 47 30 30 45 14 28 ...
 $ total.sulfur.dioxide: num  170 132 97 186 186 97 136 170 132 129 ...
 $ density             : num  1.001 0.994 0.995 0.996 0.996 ...
 $ pH                  : num  3 3.3 3.26 3.19 3.19 3.26 3.18 3 3.3 3.22 ...
 $ sulphates           : num  0.45 0.49 0.44 0.4 0.4 0.44 0.47 0.45 0.49 0.45 ...
 $ alcohol             : num  8.8 9.5 10.1 9.9 9.9 10.1 9.6 8.8 9.5 11 ...
 $ quality             : int  6 6 6 6 6 6 6 6 6 6 ...


 fixed.acidity    volatile.acidity  citric.acid     residual.sugar  
 Min.   : 3.800   Min.   :0.0800   Min.   :0.0000   Min.   : 0.600  
 1st Qu.: 6.300   1st Qu.:0.2100   1st Qu.:0.2700   1st Qu.: 1.700  
 Median : 6.800   Median :0.2600   Median :0.3200   Median : 5.200  
 Mean   : 6.855   Mean   :0.2782   Mean   :0.3342   Mean   : 6.391  
 3rd Qu.: 7.300   3rd Qu.:0.3200   3rd Qu.:0.3900   3rd Qu.: 9.900  
 Max.   :14.200   Max.   :1.1000   Max.   :1.6600   Max.   :65.800  
   chlorides       free.sulfur.dioxide total.sulfur.dioxide    density      
 Min.   :0.00900   Min.   :  2.00      Min.   :  9.0        Min.   :0.9871  
 1st Qu.:0.03600   1st Qu.: 23.00      1st Qu.:108.0        1st Qu.:0.9917  
 Median :0.04300   Median : 34.00      Median :134.0        Median :0.9937  
 Mean   :0.04577   Mean   : 35.31      Mean   :138.4        Mean   :0.9940  
 3rd Qu.:0.05000   3rd Qu.: 46.00      3rd Qu.:167.0        3rd Qu.:0.9961  
 Max.   :0.34600   Max.   :289.00      Max.   :440.0   

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6


In [66]:
# LOgistic Regression

set.seed(6501)

nobs <- nrow(white)
spl <- 0.8

shuffle_index <- sample(1:nobs)
white <- white[shuffle_index, ]

train_indices = sample(1:nobs, nobs * spl)

train = white[train_indices,] 
test = white[-train_indices,]

mod1 = lm(quality ~. - chlorides - total.sulfur.dioxide ,data = train) 
summary(mod1)

kv <- 9
ad <- 2*kv*(kv+1)/(nrow(train)-kv-1)
aic_ad1 <- round(AIC(mod1) + ad,1)
bic1 <- round(BIC(mod1),1)

print(paste('AIC = ', aic_ad1, '   BIC = ', bic1))





Call:
lm(formula = quality ~ . - chlorides - total.sulfur.dioxide, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7514 -0.5040 -0.0328  0.4623  3.1285 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)          1.403e+02  1.958e+01   7.166 9.21e-13 ***
fixed.acidity        4.748e-02  2.254e-02   2.106   0.0352 *  
volatile.acidity    -1.889e+00  1.242e-01 -15.212  < 2e-16 ***
citric.acid         -9.342e-03  1.065e-01  -0.088   0.9301    
residual.sugar       7.933e-02  7.954e-03   9.974  < 2e-16 ***
free.sulfur.dioxide  3.094e-03  7.485e-04   4.134 3.64e-05 ***
density             -1.400e+02  1.985e+01  -7.052 2.08e-12 ***
pH                   5.575e-01  1.144e-01   4.871 1.15e-06 ***
sulphates            6.075e-01  1.125e-01   5.402 6.98e-08 ***
alcohol              2.132e-01  2.616e-02   8.151 4.82e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7504 on 3908 deg

[1] "AIC =  8881.2    BIC =  8950.2"
